In [13]:
import pandas as pd

In [14]:
filepath = 'Utimas_Desvinculaciones.xlsx'
data_sheetname = 'Row data'
salarios_sheetname = 'Rangos'
managers_sheetname = 'Managers'

df = pd.read_excel(filepath, sheet_name=data_sheetname)
df_salarios = pd.read_excel(filepath, sheet_name=salarios_sheetname)
df_managers = pd.read_excel(filepath, sheet_name=managers_sheetname)

In [15]:
df.head()

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos
0,1,Romina,Brito,Marketing,2018-03-05,2018-12-31,N3,Grupo B,3,Rango 3,Rango 3,1,SI
1,2,Pedro,Lujan,Compras,2017-03-06,2021-12-31,N3,Grupo B,3,Rango 2,Rango 2,2,SI
2,3,Juan,Albornoz,IT,2018-06-04,2019-04-30,N4,Grupo C,4,Rango 4,Rango 4,5,NO
3,4,Sebastian,González,Legales,2018-05-07,2021-05-30,N2,Grupo A,4,Rango 2,Rango 2,3,SI
4,5,Franco,Rodríguez,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,1,SI


In [16]:
df_salarios.rename(columns={'desde':'Salario Desde', 'hasta':'Salario Hasta'},inplace=True)

df_salarios['Salario Promedio'] = (df_salarios['Salario Desde'] + df_salarios['Salario Hasta'])/2

df_salarios.head()

,Rango,Salario Desde,Salario Hasta,Salario Promedio
0,Rango 1,150000,200000,175000.0
1,Rango 2,200001,250000,225000.5
2,Rango 3,250001,300000,275000.5
3,Rango 4,300001,350000,325000.5
4,Rango 5,350001,400000,375000.5


In [17]:
df_managers['Manager Nombre Completo'] = df_managers['Nombre'] + ' ' + df_managers['Apellido']
df_managers.rename(columns={'Id':'Manager_Id'}, inplace=True)

# MERGE SALARIOS EMPRESA PROPIA
merged_salarios_df = pd.merge(df, 
                              df_salarios[['Rango','Salario Desde','Salario Hasta','Salario Promedio']], 
                              how='inner', 
                              left_on='Rango Salarial', 
                              right_on='Rango').drop('Rango', axis=1)
# MERGE SALARIOS COMPETENCIA
merged_salarios_df = pd.merge(merged_salarios_df, 
                              df_salarios[['Rango','Salario Promedio']], 
                              how='inner', 
                              left_on='RS competencia', 
                              right_on='Rango',
                              suffixes=('',' Competencia')).drop('Rango', axis=1)

merged_salarios_df['Empleado Nombre Completo'] = merged_salarios_df['Nombre'] + ' ' + merged_salarios_df['Apellido']

final_merge_df = pd.merge(merged_salarios_df, 
                          df_managers[['Manager_Id', 'Manager Nombre Completo']], 
                          how='inner', 
                          left_on='Manager', 
                          right_on='Manager_Id').drop(['Manager_Id','Manager','Nombre','Apellido'], axis=1)

In [18]:
final_merge_df.head()

,Id,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Realizo Cursos,Salario Desde,Salario Hasta,Salario Promedio,Salario Promedio Competencia,Empleado Nombre Completo,Manager Nombre Completo
0,1,Marketing,2018-03-05,2018-12-31,N3,Grupo B,3,Rango 3,Rango 3,SI,250001,300000,275000.5,275000.5,Romina Brito,Rolando Casas
1,5,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,SI,350001,400000,375000.5,325000.5,Franco Rodríguez,Rolando Casas
2,8,Compras,2017-01-02,2021-06-30,N3,Grupo A,4,Rango 3,Rango 3,SI,250001,300000,275000.5,275000.5,Francesca López,Marina Pitana
3,2,Compras,2017-03-06,2021-12-31,N3,Grupo B,3,Rango 2,Rango 2,SI,200001,250000,225000.5,225000.5,Pedro Lujan,Marina Pitana
4,27,Compras,2019-06-03,2022-04-30,N2,Grupo A,4,Rango 2,Rango 2,SI,200001,250000,225000.5,225000.5,Rodrigo Molinas,Marina Pitana


In [20]:
final_merge_df.to_excel('final_merge_df.xlsx', index=False)

df_salarios.to_excel('salarios.xlsx', index=False)